In [ ]:
### Mount google drive if available
try:
    from google.colab import drive
    drive.mount('/content/drive')
    drive_path = '/content/drive/MyDrive/term_paper/'
    in_colab = True
except:
    drive_path = ''
    in_colab = False

In [ ]:
### Install all dependecies

# pytorch3d
import os
import sys
import torch

need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True

if need_pytorch3d:
    if torch.__version__.startswith("1.10.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'


# smpl-x
need_smplx=False
try:
    import smplx
except ModuleNotFoundError:
    need_smplx=True

if need_smplx:
    !pip install smplx
    !git clone https://github.com/vchoutas/smplx
    %cd smplx
    !python setup.py install
    %cd ..


# cleanup
!rm -rf 1.10.0.tar.gz cub-1.10.0/

In [ ]:
import smplx
import matplotlib.pyplot as plt
from pytorch3d.structures import Meshes

from utils.plot_structures import plot_structure
from utils.smpl_to_smplx import humbi_smpl_mesh, construct_smplx_mesh, smpl2smplx
from utils.smplx_to_disps import smplx2disps
from utils.inpainting import get_disps_inpaint

In [ ]:
### Setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
subject = 122
pose = '00000001'

smplx_model_path = drive_path + 'smplx'
smplx_model = smplx.SMPLXLayer(smplx_model_path, 'neutral').to(device)

In [ ]:
global_orient, transl, body_pose, betas, scale, pose_loss, shape_loss = smpl2smplx(smplx_model, subject, pose, pose_iterations=200, shape_iterations=100)

In [ ]:
smpl_mesh = humbi_smpl_mesh(subject, pose)
smplx_mesh = construct_smplx_mesh(smplx_model, global_orient, transl, body_pose, betas, scale)

plot_structure([smpl_mesh, smplx_mesh])

In [ ]:
shape_displacements = smplx2disps(smplx_model, betas, scale)

init_verts = smplx_model.forward()['vertices'].to(device)
smplx_faces = torch.Tensor(smplx_model.faces.astype('int')).type(torch.int32).unsqueeze(0).to(device)

init_mesh = Meshes(init_verts, smplx_faces)
displaced_mesh = Meshes(init_verts + (init_mesh.verts_normals_packed() * shape_displacements.unsqueeze(1)), smplx_faces)

plot_structure([init_mesh, displaced_mesh])

In [ ]:
obj_path = drive_path + 'smplx/smplx_uv.obj'
texture_path = 'humbi_maps/00000025_humbi_body_texture/body_texture_medians/'

inpainted, mask, texture = get_disps_inpaint(subject, shape_displacements, obj_path, texture_path, mask_disps=True)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(inpainted, cmap='gray')

plt.figure(figsize=(10, 10))
plt.imshow(mask, cmap='gray')

plt.figure(figsize=(10, 10))
plt.imshow(texture)